In [1]:
"""
Nepal District Vulnerability Analysis - Data Cleaning - FINAL CORRECTED VERSION
Step 2: Clean and merge district-level data
Author: [Your Name]
Date: [Today's Date]
"""

import pandas as pd
import numpy as np
from pathlib import Path
import re

# Setup paths
RAW_PATH = Path(r'C:\Users\saurav\Downloads\SEVI_Nepal_Project\data\raw')
PROCESSED_PATH = Path(r'C:\Users\saurav\Downloads\SEVI_Nepal_Project\data\processed')
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

print("="*80)
print("DATA CLEANING - FINAL CORRECTED VERSION")
print("="*80)

# ============================================================================
# STEP 1: Load all files
# ============================================================================

def load_all_files():
    """Load all CSV files with consistent encoding"""
    csv_files = list(RAW_PATH.glob('*.csv'))
    print(f"Found {len(csv_files)} CSV files\n")
    
    dataframes = {}
    for csv_file in csv_files:
        try:
            # Try different encodings if needed
            try:
                df = pd.read_csv(csv_file, encoding='utf-8')
            except:
                df = pd.read_csv(csv_file, encoding='latin-1')
                
            dataframes[csv_file.stem] = df
            print(f"{csv_file.stem:30} {df.shape[0]:4} rows × {df.shape[1]:3} cols")
        except Exception as e:
            print(f"✗ Error loading {csv_file.name}: {e}")
    
    return dataframes

dataframes = load_all_files()

# ============================================================================
# STEP 2: Standardize district IDs from cooking_fuel.csv
# ============================================================================

print("\n" + "="*80)
print("STANDARDIZING DISTRICT IDS")
print("="*80)

# GLOBAL VARIABLE - will be used by all functions
DISTRICT_IDS = None
DISTRICT_REF = None

if 'cooking_fuel' in dataframes:
    cf_df = dataframes['cooking_fuel']
    
    # Standardize column names
    cf_df.columns = [col.strip().upper() for col in cf_df.columns]
    
    print("\nColumn names in cooking_fuel.csv:")
    print(cf_df.columns.tolist())
    
    # Find correct ID and name columns
    id_col = None
    name_col = None
    
    for col in cf_df.columns:
        if col == 'ID' or 'ID_' in col:
            id_col = col
        elif col in ['AREA', 'AREA_NAME', 'NAME', 'DISTRICT']:
            name_col = col
    
    if not id_col or not name_col:
        print("ERROR: Could not find ID or name column in cooking_fuel.csv")
        exit(1)
    
    print(f"\nUsing columns: {id_col} as ID, {name_col} as district name")
    
    # Extract districts (IDs 14 and above for Nepal districts)
    DISTRICT_REF = cf_df[cf_df[id_col] >= 14].copy()
    DISTRICT_REF = DISTRICT_REF[[id_col, name_col]].copy()
    DISTRICT_REF.columns = ['ID', 'DISTRICT_NAME']
    DISTRICT_REF = DISTRICT_REF.sort_values('ID').reset_index(drop=True)
    
    # Clean district names
    DISTRICT_REF['DISTRICT_NAME'] = DISTRICT_REF['DISTRICT_NAME'].str.strip()
    
    # Create district ID list
    DISTRICT_IDS = DISTRICT_REF['ID'].tolist()
    
    print(f"\n✓ Found {len(DISTRICT_REF)} districts")
    print(f"✓ District ID range: {min(DISTRICT_IDS)} to {max(DISTRICT_IDS)}")
    
    if len(DISTRICT_REF) == 77:
        print("✓ Perfect! All 77 districts found")
    else:
        print(f"⚠ Warning: Expected 77 districts, found {len(DISTRICT_REF)}")
    
    print("\nFirst 10 districts:")
    print(DISTRICT_REF.head(10).to_string(index=False))
    
    # Save district reference
    DISTRICT_REF.to_csv(PROCESSED_PATH / 'district_reference.csv', index=False)
    print(f"\n✓ District reference saved")
else:
    print("ERROR: cooking_fuel.csv not found - needed for district IDs")
    exit(1)

# ============================================================================
# STEP 3: Function to extract districts from any file
# ============================================================================

def clean_column_names(df):
    """Standardize column names"""
    df.columns = [col.strip().upper() for col in df.columns]
    return df

def extract_districts(df, file_name):
    """Extract district rows from any dataframe"""
    print(f"\nProcessing: {file_name}")
    print(f"  Shape: {df.shape}")
    
    # Clean column names first
    df = clean_column_names(df.copy())
    
    # Case 1: File has ID column
    if 'ID' in df.columns:
        # Filter for district IDs
        district_df = df[df['ID'].isin(DISTRICT_IDS)].copy()
        
        if len(district_df) == 0:
            print(f"  ⚠ No districts found with standard IDs")
            print(f"  Trying alternative: ID >= 14")
            district_df = df[df['ID'] >= 14].copy()
        
        print(f"  Found {len(district_df)} district rows")
        
        # Standardize name column
        name_col = None
        for col in df.columns:
            if col in ['AREA', 'AREA_NAME', 'NAME', 'DISTRICT']:
                name_col = col
                break
        
        if name_col and name_col != 'DISTRICT_NAME':
            district_df = district_df.rename(columns={name_col: 'DISTRICT_NAME'})
            
        return district_df
    
    # Case 2: File has AREA_NAME or similar (like household_amenities)
    elif 'AREA_NAME' in df.columns or 'DISTRICT' in df.columns:
        name_col = 'AREA_NAME' if 'AREA_NAME' in df.columns else 'DISTRICT'
        
        print(f"  File has {name_col} column instead of ID")
        print(f"  Will need to merge by name later")
        
        # Clean names for consistency
        df = df.copy()
        df[name_col] = df[name_col].str.strip()
        
        return df
    
    # Case 3: Cannot identify districts
    else:
        print(f"  ⚠ Cannot identify districts - no ID or name column")
        return None

# ============================================================================
# STEP 4: Process all files
# ============================================================================

print("\n" + "="*80)
print("PROCESSING ALL FILES")
print("="*80)

district_dfs = {}
files_with_ids = []
files_without_ids = []

for file_name, df in dataframes.items():
    result = extract_districts(df, file_name)
    
    if result is not None:
        district_dfs[file_name] = result
        
        if 'ID' in result.columns:
            files_with_ids.append(file_name)
        else:
            files_without_ids.append(file_name)

print(f"\n✓ Processed {len(district_dfs)} files")
print(f"  Files with ID column: {len(files_with_ids)}")
print(f"  Files without ID column: {len(files_without_ids)}")

# ============================================================================
# STEP 5: Merge all data - FIXED VERSION
# ============================================================================

print("\n" + "="*80)
print("MERGING ALL DISTRICT DATA")
print("="*80)

# Start with district reference as base
merged_df = DISTRICT_REF.copy()
print(f"Starting base: {len(merged_df)} districts")

# Function to create safe column names
def create_column_prefix(file_name):
    """Create short prefix from filename"""
    # Remove special characters, take first 3-4 chars
    clean_name = re.sub(r'[^a-zA-Z]', '', file_name)
    prefix = clean_name[:4].upper() if len(clean_name) >= 4 else clean_name.upper()
    return prefix

# Merge files WITH ID columns first
print("\nMerging files with ID columns:")
for file_name in files_with_ids:
    if file_name != 'cooking_fuel':  # Already have district names from this
        df = district_dfs[file_name]
        
        # Get columns to merge (exclude metadata)
        exclude_patterns = ['ID', 'DISTRICT_NAME', 'SEX', 'AREA_LEVEL', 
                           'AREA_TYPE', 'TOTAL_', 'AREA_']
        cols_to_merge = []
        
        for col in df.columns:
            if col not in ['ID', 'DISTRICT_NAME'] and not any(pattern in col for pattern in exclude_patterns):
                cols_to_merge.append(col)
        
        if cols_to_merge:
            # Create unique column names
            prefix = create_column_prefix(file_name)
            rename_dict = {col: f"{prefix}_{col}" for col in cols_to_merge}
            
            # Prepare DataFrame for merge
            df_to_merge = df[['ID'] + cols_to_merge].rename(columns=rename_dict)
            
            # Merge
            before_cols = len(merged_df.columns)
            merged_df = pd.merge(merged_df, df_to_merge, on='ID', how='left')
            added_cols = len(merged_df.columns) - before_cols
            
            print(f"  {file_name:25} +{added_cols:3} columns")

# Handle files WITHOUT ID columns (merge by name)
print("\nMerging files without ID columns:")
for file_name in files_without_ids:
    df = district_dfs[file_name]
    
    # Find name column
    name_col = None
    for col in ['AREA_NAME', 'DISTRICT']:
        if col in df.columns:
            name_col = col
            break
    
    if name_col:
        # Clean and standardize names
        df = df.copy()
        df[name_col] = df[name_col].str.strip().str.upper()
        
        # Filter for likely district rows (not aggregates)
        # Districts usually don't have these keywords
        exclude_keywords = ['TOTAL', 'NATIONAL', 'PROVINCE', 'REGION', 'URBAN', 'RURAL']
        mask = ~df[name_col].str.contains('|'.join(exclude_keywords), case=False, na=False)
        district_rows = df[mask].copy()
        
        if len(district_rows) > 0:
            # Get data columns
            exclude_cols = [name_col, 'AREA_TYPE', 'TOTAL']
            cols_to_merge = [col for col in district_rows.columns 
                           if col not in exclude_cols and not col.startswith('UNNAMED')]
            
            if cols_to_merge:
                # Create unique column names
                prefix = create_column_prefix(file_name)
                rename_dict = {col: f"{prefix}_{col}" for col in cols_to_merge}
                
                # Prepare for merge
                df_to_merge = district_rows[[name_col] + cols_to_merge].copy()
                df_to_merge = df_to_merge.rename(columns={name_col: 'DISTRICT_NAME', **rename_dict})
                
                # Standardize district names for merge
                merged_df['DISTRICT_NAME_UPPER'] = merged_df['DISTRICT_NAME'].str.upper()
                df_to_merge['DISTRICT_NAME_UPPER'] = df_to_merge['DISTRICT_NAME'].str.upper()
                
                # Merge
                before_cols = len(merged_df.columns)
                merged_df = pd.merge(merged_df, df_to_merge.drop('DISTRICT_NAME', axis=1), 
                                   on='DISTRICT_NAME_UPPER', how='left')
                added_cols = len(merged_df.columns) - before_cols
                
                # Clean up
                merged_df = merged_df.drop('DISTRICT_NAME_UPPER', axis=1)
                
                print(f"  {file_name:25} +{added_cols:3} columns")
        else:
            print(f"  {file_name:25} No district rows found")

print(f"\nFinal dataset shape: {merged_df.shape}")
print(f"Districts: {len(merged_df)}")
print(f"Total features: {len(merged_df.columns)}")

# ============================================================================
# STEP 6: Data quality checks and save - IMPROVED
# ============================================================================

print("\n" + "="*80)
print("DATA QUALITY CHECKS")
print("="*80)

# 1. Check for duplicate districts
duplicate_ids = merged_df[merged_df.duplicated('ID', keep=False)]
if len(duplicate_ids) > 0:
    print(f"⚠ WARNING: Found {len(duplicate_ids)} duplicate district IDs")
    print(duplicate_ids[['ID', 'DISTRICT_NAME']].head())
else:
    print("✓ No duplicate district IDs")

# 2. Check all 77 districts are present
if len(merged_df) == 77:
    print("✓ All 77 districts present")
else:
    print(f"⚠ Missing districts: Expected 77, found {len(merged_df)}")
    missing_ids = set(DISTRICT_IDS) - set(merged_df['ID'].tolist())
    if missing_ids:
        print(f"  Missing IDs: {sorted(missing_ids)}")

# 3. Check for missing values
missing = merged_df.isnull().sum()
missing_cols = missing[missing > 0]

if len(missing_cols) > 0:
    print(f"\n⚠ Columns with missing values: {len(missing_cols)}")
    
    # Calculate overall missing percentage
    total_cells = merged_df.shape[0] * merged_df.shape[1]
    missing_cells = missing.sum()
    missing_pct = (missing_cells / total_cells) * 100
    
    print(f"  Overall missing data: {missing_pct:.1f}%")
    
    # Show columns with >20% missing
    high_missing = []
    for col, count in missing_cols.items():
        pct = count / len(merged_df) * 100
        if pct > 20:
            high_missing.append((col, count, pct))
    
    if high_missing:
        print(f"\n  Columns with >20% missing values:")
        for col, count, pct in sorted(high_missing, key=lambda x: x[2], reverse=True)[:10]:
            print(f"    {col:35} {count:3} missing ({pct:5.1f}%)")
else:
    print("✓ No missing values!")

# 4. Check column naming consistency
print(f"\nColumn naming patterns:")
prefixes = {}
for col in merged_df.columns:
    if '_' in col:
        prefix = col.split('_')[0]
        prefixes[prefix] = prefixes.get(prefix, 0) + 1

print(f"  Unique prefixes: {len(prefixes)}")
print(f"  Most common prefixes:")
for prefix, count in sorted(prefixes.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"    {prefix:10} : {count:3} columns")

# ============================================================================
# STEP 7: Save the cleaned data
# ============================================================================

print("\n" + "="*80)
print("SAVING CLEANED DATA")
print("="*80)

# Save merged dataset
output_path = PROCESSED_PATH / 'nepal_districts_cleaned.csv'
merged_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✓ Cleaned data saved to: {output_path}")

# Save a simplified version
district_list = merged_df[['ID', 'DISTRICT_NAME']].copy()
district_list.to_csv(PROCESSED_PATH / 'district_list.csv', index=False)
print(f"✓ District list saved")

# Create and save comprehensive summary
summary_path = PROCESSED_PATH / 'cleaning_summary.txt'
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("NEPAL DISTRICT DATA CLEANING - COMPREHENSIVE SUMMARY\n")
    f.write("="*70 + "\n\n")
    
    f.write(f"Data Shape: {merged_df.shape[0]} districts × {merged_df.shape[1]} features\n")
    f.write(f"Total Files Processed: {len(district_dfs)}\n")
    f.write(f"Files with ID columns: {len(files_with_ids)}\n")
    f.write(f"Files without ID columns: {len(files_without_ids)}\n\n")
    
    f.write("-"*70 + "\n")
    f.write("DATA QUALITY METRICS\n")
    f.write("-"*70 + "\n")
    f.write(f"Missing values: {missing_cells if 'missing_cells' in locals() else 0} cells\n")
    if 'missing_pct' in locals():
        f.write(f"Missing percentage: {missing_pct:.1f}%\n")
    f.write(f"Duplicate districts: {len(duplicate_ids)}\n")
    f.write(f"Expected districts: 77, Found: {len(merged_df)}\n\n")
    
    f.write("-"*70 + "\n")
    f.write("COLUMN CATEGORIES\n")
    f.write("-"*70 + "\n")
    
    # Categorize columns
    categories = {
        'Housing': lambda x: any(kw in x for kw in ['WALL', 'ROOF', 'FLOOR', 'FOUNDATION', 'HOUSE']),
        'Water/Sanitation': lambda x: any(kw in x for kw in ['WATER', 'TOILET', 'BATH', 'DRI_', 'TOI_']),
        'Energy': lambda x: any(kw in x for kw in ['FUEL', 'COOKING', 'LIGHT', 'ELECTRICITY', 'ENERGY']),
        'Amenities': lambda x: any(kw in x for kw in ['AMENITY', 'RADIO', 'TV', 'PHONE', 'INTERNET', 'COMPUTER']),
        'Demographics': lambda x: any(kw in x for kw in ['CHILDREN', 'POPULATION', 'AGE', 'SEX', 'MARITAL']),
        'Education': lambda x: any(kw in x for kw in ['EDUCATION', 'SCHOOL', 'LITERACY', 'DEGREE', 'ATTAINMENT']),
        'Employment': lambda x: any(kw in x for kw in ['WORK', 'OCCUPATION', 'EMPLOYMENT', 'INDUSTRY', 'JOB']),
        'Metadata': lambda x: x in ['ID', 'DISTRICT_NAME', 'AREA_TYPE', 'AREA_LEVEL']
    }
    
    categorized = {cat: [] for cat in categories.keys()}
    uncategorized = []
    
    for col in merged_df.columns:
        categorized_flag = False
        for cat, test_func in categories.items():
            if test_func(col.upper()):
                categorized[cat].append(col)
                categorized_flag = True
                break
        if not categorized_flag:
            uncategorized.append(col)
    
    for cat, cols in categorized.items():
        if cols:
            f.write(f"\n{cat} ({len(cols)} features):\n")
            for col in sorted(cols)[:15]:  # Show first 15
                f.write(f"  - {col}\n")
            if len(cols) > 15:
                f.write(f"    ... and {len(cols)-15} more\n")
    
    if uncategorized:
        f.write(f"\nUncategorized ({len(uncategorized)} features):\n")
        for col in sorted(uncategorized)[:20]:
            f.write(f"  - {col}\n")
        if len(uncategorized) > 20:
            f.write(f"    ... and {len(uncategorized)-20} more\n")
    
    f.write("\n" + "="*70 + "\n")
    f.write("DISTRICT LIST\n")
    f.write("="*70 + "\n\n")
    for _, row in merged_df[['ID', 'DISTRICT_NAME']].iterrows():
        f.write(f"{row['ID']:3} : {row['DISTRICT_NAME']}\n")

print(f"✓ Summary saved to: {summary_path}")

print("\n" + "="*80)
print("SCRIPT COMPLETE - READY FOR FEATURE ENGINEERING")
print("="*80)
print("\nNEXT STEP:")
print("1. Open 03_feature_engineering.py")
print("2. Key tasks:")
print("   - Convert counts to percentages")
print("   - Create vulnerability indices")
print("   - Handle any remaining missing data")
print(f"\nYour cleaned data is ready at: {output_path}")

DATA CLEANING - FINAL CORRECTED VERSION
Found 16 CSV files

below_secondary_education       222 rows ×  24 cols
children_living_arrangement     231 rows ×  14 cols
cooking_fuel                     90 rows ×   9 cols
drinking_watersource             90 rows ×  11 cols
educational_attainment          270 rows ×  16 cols
educational_field_distribution  267 rows ×  18 cols
floor_type                       90 rows ×   8 cols
foundation_type                  90 rows ×   7 cols
household_amenities              90 rows ×  19 cols
housing_ownership                90 rows ×   7 cols
lighting_source                  90 rows ×   7 cols
months_worked                   270 rows ×   9 cols
population_occupation           270 rows ×  16 cols
roof_type                        90 rows ×  10 cols
toilet_facility                  90 rows ×   7 cols
wall_materials                   90 rows ×  10 cols

STANDARDIZING DISTRICT IDS

Column names in cooking_fuel.csv:
['ID', 'AREA', 'WOOD_FIREWOOD', 'LPG_GAS', 'E